## Database Backend - Influx

### Imports

In [ ]:
# -*- coding: utf-8 -*-
### from https://influxdb-python.readthedocs.io/en/latest/examples.html
from influxdb import InfluxDBClient
import os
import sys

## Get Client

In [ ]:

def main(host='86.175.237.116', port=8086):
    """Instantiate a connection to the InfluxDB."""
    user = 'admin'
    password = 'admin'
    dbname = 'metrics'
    # dbuser = 'smly'
    # dbuser_password = 'my_secret_password'
    # SELECT mean("load1") FROM "default"."system" WHERE $timeFilter GROUP BY time($__interval) fill(null)
    query = 'select mean("load1") from "system";'
    query_where = 'select Int_value from cpu_load_short where host=$host;'
    bind_params = {'host': 'server01'}
    json_body = [
            {
                'measurement': 'data',
                'tags': {
                    'device': topic,
                    #"domain": "sensor"
                    },
                'fields': my_dict
            }
        ]
    
    client = InfluxDBClient(host, port, user, password, dbname)
    print("DBs:", client.get_list_database())

    # print("Create database: " + dbname)
    # client.create_database(dbname)

    # print("Create a retention policy")
    # client.create_retention_policy('awesome_policy', '3d', 3, default=True)

    # print("Switch user: " + dbuser)
    # client.switch_user(dbuser, dbuser_password)

    # print("Write points: {0}".format(json_body))
    # client.write_points(json_body)

    print("Querying data: " + query)
    result = client.query(query)

    print("Result: {0}".format(result))

    # print("Querying data: " + query_where)
    # result = client.query(query_where, bind_params=bind_params)

    # print("Result: {0}".format(result))

    # print("Switch user: " + user)
    # client.switch_user(user, password)

    # print("Drop database: " + dbname)
    # client.drop_database(dbname)


def parse_args():
    """Parse the args."""
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()


if __name__ == '__main__':
    args = parse_args()
    main(host=args.host, port=args.port)


### Check packages

In [ ]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "influxdb_client"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

## Read in credentials

Sets a dictionay with:
- 'usr': "USER_NAME",
- 'pwd': "USER_PASSWORD",
- 'host': "HOST",
- 'port': PORT


In [ ]:
### set up Kenny's credentials (ignore if user!=Kenny)
credDir=os.getcwd()
if os.path.isdir(credDir):
    print("directory found:",credDir)
    sys.path.insert(1, credDir)
    import connectionDetails
    credDict=connectionDetails.GetInflux1Credentials()
    print(credDict)
    print("done.")
else:
    print("no directory found:",credDir)

## Query Influx

### get API client

In [ ]:
### set client to get access to influx
client = InfluxDBClient(credDict['host'], credDict['port'], credDict['user'], credDict['password']) #, dbname)


### check existing databases

In [ ]:
### set bucket api
print("DBs:", client.get_list_database())

### Query data

Example query from influxdb GUI (replace all-caps):
<code>
from(bucket: BUCKET_NAME)
  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
  |> filter(fn: (r) => r["_measurement"] == DATA_NAME)
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")
</code>

In [ ]:
### reset client with chosen database
dbName="metrics"
client = InfluxDBClient(credDict['host'], credDict['port'], credDict['user'], credDict['password'], dbName)


In [ ]:
### set query str
query = 'select mean("load1") from "system";'

print("Querying data: " + query)
result = client.query(query)

print("Result: {0}".format(result))

### Write data

In [ ]:
json_body = [
        {
            'measurement': 'test',
            'tags': {
                'origin': "notebook"
                },
            'fields': None
        }
    ]
print("Write points: {0}".format(json_body))
client.write_points(json_body)